# From ChiP to Core Regulatory Circuitry.
Jérémie Kalfon
@jkobject

[www.jkobject.com](https://www.jkobject.com)

in partnership with the Broad Institute

thanks to young Lab @ MIT

In [26]:
from Bio import SeqIO
import os
path_to_rose = "/Users/jeremie/Documents/logiciels/rose"
mypath= "datatosend"
os.chdir(path_to_rose)
import ROSE_utils
a = ["/Users/jeremie/Documents/Projects/BroadInstitute/CHiP2CRC"] #! pwd
a = a[0]
%load_ext autoreload
%autoreload 2

### you might need to install an extention to curl to download from an sftp server. otherwise reinstall the latest version of curl with libssh

on mac: 
`brew install curl --with-libssh2=/usr/local`
after installing [conda]( 
```
conda config --add channels defaults
conda config --add channels bioconda
conda config --add channels conda-forge
```
then
`conda config install bowtie2 && pip install MACS2`
then `git clone` and provide path to [ROSE](https://bitbucket.org/young_computation/rose/src) and [CRCMapper](https://bitbucket.org/young_computation/crcmapper)

for CRC mapper you need [meme](http://meme-suite.org/doc/install.html?man_type=web)
Not every version will work as they do not contain the same functions unfortunatly. I have found version **4.12.0** to work.

In [ ]:
mypath= "datatosend"
! curl -k "sftp://bioinfodata.wistar.upenn.edu/datatosend.tar.gz" --user [NAME] > $mypath.tar.gz


## the call might stop for many reasons. in this case just run this function below to continue it from where you started as the -R option is not working well with curl and sftp

In [ ]:
def notfinished():
    s = os.path.getsize(mypath+'.tar.gz')
    ! curl -C $s -k "sftp://bioinfodata.wistar.upenn.edu/datatosend.tar.gz" --user [NAME] > $mypath1.tar.gz
    ! cat $mypath.tar.gz $mypath1.tar.gz > $mypath2.tar.gz
    ! rm $mypath.tar.gz $mypath1.tar.gz
    ! mv $mypath2.tar.gz $mypath.tar.gz

### here we prepare the libs and the alignments with the hg19/37 version of the genome as it is the best one currently working with CRCmapper

In [ ]:
! gzip -d $mypath.tar.gz
! curl ftp://ftp.ensembl.org/pub/grch37/current/fasta/homo_sapiens/dna/Homo_sapiens.GRCh37.dna.toplevel.fa.gz > reference.fa.gz
! gzip -d reference.fa.gz
! mkdir reference
os.chdir("reference")
! wget --timestamping 'ftp://hgdownload.cse.ucsc.edu/goldenPath/hg19/chromosomes/chr{1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,X,Y}.fa.gz'
! gzip -d *.fa.gz

### some visualization

In [ ]:
for i, rec in enumerate(SeqIO.parse(mypath+"/54-mp457-PDX34-H3K27ac-r2_S7_R1.fastq", "fastq")):
    print(rec)
    if i > 10:
        break    

## here we can see the -logp that the base call was incorrect (called phred quality/sanger quality...) 

In [ ]:
for i, rec in enumerate(SeqIO.parse(mypath+"/54-mp457-PDX34-H3K27ac-r2_S7_R1.fastq", "fastq")):
    print(rec.seq)
    if i > 10:
        break

In [ ]:
! MACS2 callpeak -h

In [ ]:
! bowtie2 -h

### Let's build the indexes with bowtie2  

In [ ]:
! bowtie2-build $mypath/reference.fa index --threads 8
#maybe try to output bampe files next time (and with higher sensibility) for macs2 to read the paired end information

### we then produce the alignments

here we use the paired end alignments

In [ ]:
! bowtie2 -x index -1 $mypath/54-mp457-PDX34-H3K27ac-r2_S7_R1.fastq.gz -2 $mypath/54-mp457-PDX34-H3K27ac-r2_S7_R2.fastq.gz -S $mypath/mapped.sam --threads 8 -t

### we can then use macs2 for the peak calling 

In [ ]:
! MACS2 callpeak -f BAMPE -t $mypath/mapped.sam --outdir "peak_called_bampe"
# 

### Some visualization...

In [ ]:
! Rscript peak_called/NA_model.r
from IPython.display import IFrame
IFrame("./peak_called/NA_model.pdf", width=600, height=300)

### finally we need some file transformation for rose to be able to process everything well

we transform the outputed bed files into gff for rose to be able to process

In [ ]:
import ROSE_utils
ROSE_utils.bedToGFF(bed="/Users/jeremie/Documents/Projects/BroadInstitute/CHiP2CRC/peak_called/NA_summits.bed",output="/Users/jeremie/Documents/Projects/BroadInstitute/CHiP2CRC/peak_called/NA_summits.gff")

we then sort and index the mapped reads

In [ ]:
! samtools sort --threads 6 $mypath/mapped.sam > $mypath/mapped_sorted.bam 

we erase the now useless file

In [ ]:
! rm $mypath/mapped.sam

we sort the reads

In [ ]:
! samtools index -@ 7 $mypath/mapped_sorted.bam

### finally we have to switch folders and then call rose
rose will now stitch the enhancers to find the superenhancers in this cell line 

In [ ]:
import os
a = ['/Users/jeremie/Documents/Projects/BroadInstitute/CHiP2CRC'] #! pwd
a = a[0]
path_to_rose = "/Users/jeremie/Documents/logiciels/rose"
os.chdir(path_to_rose)
! python ROSE_main.py -g hg19 -i $a/peak_called/NA_summits.gff -r $a/$mypath/mapped_sorted.bam -o $a/superenhancers

### We can now create a map of the core regulatory circuitry of this cell line!

they were two lines to change in crcmapper to make it work...
```python
def createExpressionDict(annotationFile, projectFolder, projectName, refseqToNameDict, expressionTable):
```
-----
line 98

----

and 
```python
for line in expressionTable[1:]:
    trid = line[0]
    geneName = refseqToNameDict[trid]
    if len(expressionTable[1]) == 3:  # when expressionTable is an output from bamToGFF.py
        exp = float(line[2])
    else:  # when expressionTable is passed as an option (2 columns)
        exp = float(line[1])
```
----
line 120 
creating syntax error

----

and
```python
self._chr = self.__chrDict[chr] if self.__chrDict[chr][:3] == 'chr' else 'chr' + self.__chrDict[chr]
```
----
in utils.py line 44
to accomodate the potential variation in name in bam output files (chr# or #) 

---

and
```python
382:done = False
    for line in fimoTable[1:]:
        source = motifDatabaseDict[line[0]]
        if not done:
            for p, s in enumerate(line):
                if '|' in s:
                    pos = p
                    done = True
                    break

        region = line[pos].split('|')
403:if (region[1], int(region[2]) + int(line[pos + 1]), int(region[2]) + int(line[pos + 2])) not in motifDictSE[source]:

405:motifDictSE[source].append((region[1], int(region[2]) + int(line[pos + 1]), int(region[2]) + int(line[pos + 1])))
```
---
to accomodate for the potential different positions of the fimo table output according to different `meme` version

---




In [51]:
path_to_crcmapper = "/Users/jeremie/Documents/logiciels/crcmapper"
os.chdir(path_to_crcmapper)
from CRCmapper import findCRC
print a, mypath
findCRC(
    fasta=a+"/"+mypath+"/reference/", 
    genome="HG19", 
    enhancers=a+"/superenhancers/NA_summits_AllEnhancers.table.txt", 
    bam=a+"/"+mypath+"/mapped_sorted.bam", 
    subpeaks=a+"/peak_called/NA_summits.bed", 
    name="CRC", 
    output=a+"/crcmapper", 
    activity=a+"/crcmappermatrix.gff",
    expCutoff=33, extension=500, Enumber="supers")
! mkdir CRC
! mv crcmapper* CRC/

/Users/jeremie/Documents/Projects/BroadInstitute/CHiP2CRC datatosend
CREATING SUPER-ENHANCER LOCUS COLLECTION
CREATING EXPRESSION DICTIONARY
Expression cutoff: 0.0
FINDING CANDIDATE TFs
Number of candidate TFs: 27
CREATE CANDIDATE TFs AND SE TABLE
MAKE FASTA
MOTIF SEARCH
Number of annotated candidate TFs that have motifs: 14
fimo --motif 'JASPAR2014.MA0102.3' --motif 'Transfac.V$CFOS_Q6' --motif 'JASPAR2014.MA0476.1' --motif 'Transfac.V$JUNB_Q6' --motif 'JASPAR2014.MA0490.1' --motif 'Transfac.V$KLF3_Q3' --motif 'Transfac.V$KLF7_04' --motif 'Transfac.V$KLF7_03' --motif 'Uniprobe.UP00093_1' --motif 'Uniprobe.UP00093_2' --motif 'Transfac.V$MEF2D_01' --motif 'Jolma2013.MEF2D_DBD' --motif 'Transfac.V$MNT_01' --motif 'Jolma2013.MNT_DBD' --motif 'Transfac.V$MZF1_02' --motif 'Transfac.V$MZF1_01' --motif 'Transfac.V$MZF1_Q5' --motif 'JASPAR2014.MA0056.1' --motif 'JASPAR2014.MA0057.1' --motif 'JASPAR2014.MA0107.1' --motif 'Transfac.V$RFX1_02' --motif 'Transfac.V$RFX1_01' --motif 'Transfac.V$EFC_

In [ ]:
! fasta-get-markov

## an automated version

In [ ]:
mypath = "datatosend"
refs = [30,31,32,33,34]
files = {}
path_to_rose = "/Users/jeremie/Documents/logiciels/rose"
path_to_crcmapper = "/Users/jeremie/Documents/logiciels/crcmapper"
a = ! pwd
a = a[0]
os.chdir(path_to_rose)
import ROSE_utils
os.chdir(a)
for ref in refs
    for r in os.listdir(mypath):
        if r[:2] in refs:
            lane = "R1" if "_R1" in r else "R2"
            files[lane] = r
            ! bowtie2 -x index -1 $mypath/$files["R1"] -2 $mypath/$files["R2"] -S $mypath/mapped.sam --threads 8 -t
            ! MACS2 callpeak -t $mypath/mapped.sam --outdir peak_called_$ref
            ! Rscript peak_called_$ref/NA_model.r
            ! samtools sort --threads 6 $mypath/mapped.sam > $mypath/mapped_sorted_$ref.bam 
            ! rm $mypath/mapped.sam
            ! samtools index -@ 7 $mypath/mapped_sorted_$ref.bam 
            os.chdir(path_to_rose)
            ROSE_utils.bedToGFF(bed=a + "/peak_called_"+ref+"/NA_summits.bed",output=a + "/peak_called_"+ ref +"/NA_summits.gff")
            ! python ROSE_main.py -g hg19 -i $a/peak_called_$ref/NA_summits.gff -r $a/$mypath/mapped_sorted_$ref.bam -o $a/superenhancers_$ref
            os.chdir(path_to_crcmapper)
            ! python CRCmapper.py -e $a/superenhancers_$ref/NA_summits_AllEnhancers.table.txt -b $a/$mypath/mapped_sorted_$ref.bam -g hg19 -f $a/$mypath/reference  -s $a/peak_called_$ref/NA_summits.bed -n CRC -o $a/crcmapper_$ref
            os.chdir(a)